### Data Cleaning

Criminal Justice - Survey of Inmates

In [1]:
from __future__ import division

import pandas as pd
import numpy as np
import copy

pd.set_option('max_columns', 100)

In [2]:
with open('inmate_state.csv') as datafile:
    dfs = pd.read_csv(datafile)
    
data = dfs.ix[:,1:]

> Look at criminals currently serving time.

> Get offense codes. Categorize by type of crime.

In [3]:
data2 = copy.deepcopy(data[data['V0083'] == 1])

In [4]:
off_cols = ['V0094', 'V0096', 'V0098', 'V0100', 'V0102',
            'V0115', 'V0117', 'V0119', 'V0121', 'V0123']

offenses = data2[off_cols]

data2['code'] = offenses[offenses.columns].min(axis=1)

data3 = copy.deepcopy(data2[data2['code'] != 999])

len(data3), len(data2)

(8099, 14290)

In [5]:
codes = data3['code']

In [6]:
violent_crime = []
for code in codes:
    if code <= 180:
        violent = 1
    else:
        violent = 0
    violent_crime.append(violent)
    
property_crime = []
for code in codes:
    if 190 <= code < 340:
        prop = 2
    else:
        prop = 0
    property_crime.append(prop)
    
drug_crime = []
for code in codes:
    if 340 <= code < 480:
        drug = 3
    else:
        drug = 0
    drug_crime.append(drug)
    
puborder_crime = []
for code in codes:
    if 480 <= code < 673:
        order = 4
    else:
        order = 0
    puborder_crime.append(order)
    
other_crime = []
for code in codes:
    if code >= 673:
        other = 5
    else:
        other = 0
    other_crime.append(other)

In [7]:
violent_crime.count(1) / len(violent_crime)

0.5761205087047784

In [8]:
crime_types = {'violent': violent_crime, 
               'property': property_crime, 
               'drug': drug_crime, 
               'publicorder': puborder_crime, 
               'other': other_crime}

for crime, vals in crime_types.items():
    data3[crime] = vals

data3['crime_type'] = data3[crime_types.keys()].max(axis=1)

data3.crime_type.value_counts()

1    4666
3    1570
2    1244
4     487
5     132
dtype: int64

> Build dataframe to use for prediction

In [9]:
data4 = copy.deepcopy(data3[['violent', 'crime_type', 'V0001', 'V0005', 'V0014']])

data4.columns = ['violent', 'crime_type', 'unique_id', 'gender', 'age']

In [10]:
race = copy.deepcopy(data3[['V0037', 'V0038', 'V0039', 'V0040', 'V0041', 
                            'V0042', 'V0043']])

data4['race'] = race[race.columns].min(axis=1)

data4['race'] = data4.race.replace(1, 
                                'white').replace(2, 
                                'black').replace(3, 
                                'native_am').replace(4, 
                                'asian').replace(5, 
                                'other').replace(6, 
                                'other').replace(7, 
                                'race_unknown').replace(8, 
                                'race_unknown').replace(9, 
                                'race_unknown')

In [11]:
data4['gender'] = data4.gender.replace(1, 'male').replace(2, 'female')

> Find more features

In [12]:
off_cols = ['V1199', 'V1201', 'V1203', 'V1205', 'V1207']

offenses = data3[off_cols]

data3['code2'] = offenses[offenses.columns].min(axis=1)

codes = data3['code2']

first_violent = []
for code in codes:
    if code <= 180:
        violent = 1
    else:
        violent = 0
    first_violent.append(violent)
    
data3['violent_first'] = first_violent

In [13]:
cols_to_add = ['V0058', 'V0059', 'V0126', 'V0131', 'V1197', 'V1198', 'violent_first', 
               'V1209', 'V1296', 'V1295', 'V1740', 'V1742', 'V1743', 'V1747', 'V1809', 
               'V1857', 'V1860', 'V1863', 'V1865', 'V1868', 'V1873', 'V1891', 'V1892', 
               'V1897', 'V1935', 'V1983', 'V1986', 'V2050', 'V2052', 'V2055', 'V2057', 
               'V2058', 'V2059', 'V2060', 'V2061', 'V2062']

col_names = ['marital_stat', 'is_military', 'year_arrested', 'year_admitted', 
             'ntimes_arrested', 'age_arrest_first', 'violent_first', 'probation_yn', 
             'ntimes_incarc_adult', 'ntimes_incarc_juv', 'highest_grade_attend', 
             'has_GED', 'has_HSD', 'had_job', 'public_assist_prior', 
             'monthly_income_prior', 'family_makeup', 'child_caretaker_welfare', 
             'child_caretaker_substabuse', 'parents_servetime', 'family_servetime', 
             'commit_crime_juv', 'age_crime_first', 'forcible_sex', 'phys_abuse', 
             'age_drink_first', 'drink_frequency', 'used_heroin', 'used_meth', 
             'used_barbiturates', 'used_crack', 'used_cocaine', 'used_pcp', 'used_ecstacy', 
             'used_lsd', 'used_pot']

for num_key, name_key in zip(cols_to_add, col_names):
    data4[name_key] = data3[num_key]

In [14]:
data4.describe()

,violent,crime_type,unique_id,age,marital_stat,is_military,year_arrested,year_admitted,ntimes_arrested,age_arrest_first,violent_first,probation_yn,ntimes_incarc_adult,ntimes_incarc_juv,highest_grade_attend,has_GED,has_HSD,had_job,public_assist_prior,monthly_income_prior,family_makeup,child_caretaker_welfare,child_caretaker_substabuse,parents_servetime,family_servetime,commit_crime_juv,age_crime_first,forcible_sex,phys_abuse,age_drink_first,drink_frequency,used_heroin,used_meth,used_barbiturates,used_crack,used_cocaine,used_pcp,used_ecstacy,used_lsd,used_pot
count,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000
mean,0.576121,1.786887,6971.199778,36.192863,3.755896,1.901346,2076.949747,2109.501543,50.513767,38.543400,0.140635,3.074824,64.448450,63.551426,11.666502,4.269293,6.471663,1.470058,3.348932,21.815533,2.836029,1.854056,1.767255,1.920978,1.750463,3.572910,54.609828,1.942832,1.861217,31.354612,4.929497,1.910730,1.868008,1.953081,1.847636,1.695271,1.954315,1.975429,1.866156,1.358192
std,0.494202,1.054467,4088.531894,10.993424,1.546130,0.391654,791.206862,934.150008,209.059822,34.883753,0.347666,3.270822,46.324149,47.221089,8.471884,3.600919,3.426904,1.263675,2.998104,34.531434,9.217202,1.192898,0.913273,0.924237,0.978573,3.562789,42.605772,0.798131,0.789535,32.478136,3.510819,0.806284,0.829791,0.786570,0.841618,0.886490,0.785857,0.773265,0.830610,0.899486
min,0.000000,1.000000,3.000000,16.000000,1.000000,1.000000,1959.000000,1960.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,0.000000,1.000000,3396.000000,27.000000,3.000000,2.000000,1996.000000,1997.000000,1.000000,16.000000,0.000000,1.000000,3.000000,1.000000,10.000000,1.000000,2.000000,1.000000,2.000000,4.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,14.000000,2.000000,2.000000,14.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,2.000000,2.000000,1.000000
50%,1.000000,1.000000,6890.000000,35.000000,5.000000,2.000000,2000.000000,2001.000000,2.000000,19.000000,0.000000,1.000000,99.000000,99.000000,11.000000,2.000000,9.000000,1.000000,2.000000,7.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,19.000000,2.000000,2.000000,17.000000,4.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000
75%,1.000000,3.000000,10348.500000,43.000000,5.000000,2.000000,2002.000000,2003.000000,6.000000,45.500000,0.000000,2.000000,99.000000,99.000000,12.000000,9.000000,9.000000,2.000000,2.000000,11.000000,2.000000,2.000000,2.000000,2.000000,2.000000,9.000000,99.000000,2.000000,2.000000,21.000000,9.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
max,1.000000,5.000000,14498.000000,84.000000,8.000000,8.000000,9999.000000,9999.000000,999.000000,99.000000,1.000000,9.000000,99.000000,99.000000,99.000000,9.000000,9.000000,9.000000,9.000000,99.000000,99.000000,9.000000,9.000000,9.000000,9.000000,9.000000,99.000000,9.000000,9.000000,99.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000


> Replace missing values and reassign categorical variables

In [15]:
data4['has_HSD'] = data4['has_HSD'].replace(1, 0).replace(2, 1)

def replace_NaNs1(lst):
    for col in lst:
        data4[col] = data4[col].replace(2, 0).replace(7, 0).replace(8, 0).replace(9, 0)

nans_list1 = ['probation_yn', 'has_GED', 'has_HSD', 'had_job', 'public_assist_prior', 
              'child_caretaker_welfare', 'child_caretaker_substabuse', 'parents_servetime', 
              'is_military', 'family_servetime', 'commit_crime_juv', 'forcible_sex', 
              'phys_abuse', 'used_heroin', 'used_meth', 'used_barbiturates', 'used_crack',
              'used_cocaine', 'used_pcp', 'used_ecstacy', 'used_lsd', 'used_pot']

replace_NaNs1(nans_list1)

In [16]:
nans_list2 = ['highest_grade_attend', 'age_arrest_first', 'age_crime_first', 
              'age_drink_first']

def replace_NaNs2(lst):
    for col in lst:
        data4[col] = data4[col].replace(99, 
                                        np.NaN).replace(98, 
                                        np.NaN).replace(97, 
                                        np.NaN).replace(999, 
                                        np.NaN).replace(998,
                                        np.NaN).replace(997,
                                        np.NaN)

replace_NaNs2(nans_list2)

data4['highest_grade_attend'] = data4['highest_grade_attend'].replace(19, np.NaN)

In [17]:
data4 = data4.fillna(data4.mean())

In [18]:
nans_list3 = ['ntimes_incarc_juv', 'ntimes_incarc_adult', 'ntimes_arrested']

replace_NaNs2(nans_list3)

data4 = data4.fillna(data4.median())

In [19]:
data4['marital_stat'] = data4.marital_stat.replace(1, 
                                                'married').replace(2, 
                                                'widowed').replace(3, 
                                                'divorced').replace(4, 
                                                'separated').replace(5, 
                                                'never_marr').replace(7, 
                                                'mar_unknown').replace(8, 
                                                'mar_unknown').replace(9,
                                                'mar_unknown')

In [20]:
data4['monthly_income_prior'] = data4['monthly_income_prior'].replace(1, 
                                                        100).replace(2,
                                                        300).replace(3,
                                                        500).replace(4,
                                                        700).replace(5,
                                                        900).replace(6,
                                                        1100).replace(7,
                                                        1350).replace(8,
                                                        1750).replace(9,
                                                        2250).replace(10,
                                                        3750).replace(11,
                                                        6250).replace(12,
                                                        7500).replace(97,
                                                        np.NaN).replace(98,
                                                        0).replace(99, 0)

data4 = data4.fillna(data4.mean())

In [21]:
data4.describe()

,violent,crime_type,unique_id,age,is_military,year_arrested,year_admitted,ntimes_arrested,age_arrest_first,violent_first,probation_yn,ntimes_incarc_adult,ntimes_incarc_juv,highest_grade_attend,has_GED,has_HSD,had_job,public_assist_prior,monthly_income_prior,family_makeup,child_caretaker_welfare,child_caretaker_substabuse,parents_servetime,family_servetime,commit_crime_juv,age_crime_first,forcible_sex,phys_abuse,age_drink_first,drink_frequency,used_heroin,used_meth,used_barbiturates,used_crack,used_cocaine,used_pcp,used_ecstacy,used_lsd,used_pot
count,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000,8099.000000
mean,0.576121,1.786887,6971.199778,36.192863,0.108038,2076.949747,2109.501543,4.173602,18.968929,0.140635,0.547228,2.150019,0.232004,10.873358,0.280899,0.298308,0.700951,0.186196,1690.210014,2.836029,0.348438,0.326707,0.193357,0.362020,0.526732,13.827962,0.136437,0.206198,15.962250,4.929497,0.165082,0.208544,0.123472,0.229658,0.381282,0.122237,0.101124,0.210396,0.719101
std,0.494202,1.054467,4088.531894,10.993424,0.310448,791.206862,934.150008,6.656813,5.976720,0.347666,0.497795,2.024142,0.895439,2.474814,0.449466,0.457544,0.457870,0.389289,1946.840411,9.217202,0.476505,0.469038,0.394955,0.480614,0.499316,2.374618,0.343273,0.404600,3.861386,3.510819,0.371278,0.406293,0.328999,0.420639,0.485731,0.327580,0.301511,0.407615,0.449466
min,0.000000,1.000000,3.000000,16.000000,0.000000,1959.000000,1960.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,3396.000000,27.000000,0.000000,1996.000000,1997.000000,1.000000,16.000000,0.000000,0.000000,2.000000,0.000000,10.000000,0.000000,0.000000,0.000000,0.000000,300.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.827962,0.000000,0.000000,14.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,6890.000000,35.000000,0.000000,2000.000000,2001.000000,2.000000,18.968929,0.000000,1.000000,2.000000,0.000000,11.000000,0.000000,0.000000,1.000000,0.000000,1100.000000,2.000000,0.000000,0.000000,0.000000,0.000000,1.000000,13.827962,0.000000,0.000000,15.962250,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,1.000000,3.000000,10348.500000,43.000000,0.000000,2002.000000,2003.000000,5.000000,19.000000,0.000000,1.000000,2.000000,0.000000,12.000000,1.000000,1.000000,1.000000,0.000000,2250.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,14.000000,0.000000,0.000000,17.000000,9.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,5.000000,14498.000000,84.000000,1.000000,9999.000000,9999.000000,80.000000,70.000000,1.000000,1.000000,80.000000,15.000000,18.000000,1.000000,1.000000,1.000000,1.000000,7500.000000,99.000000,1.000000,1.000000,1.000000,1.000000,1.000000,37.000000,1.000000,1.000000,55.000000,9.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [22]:
data4['family_makeup'] = data4['family_makeup'].replace(1, 
                                        'both').replace(2,
                                        'mother').replace(3,
                                        'father').replace(4,
                                        'grandparents').replace(5,
                                        'other_rel').replace(6,
                                        'friends').replace(7,
                                        'foster').replace(8,
                                        'institution').replace(9,
                                        'family_unknown').replace(97,
                                        'family_unknown').replace(98,
                                        'family_unknown').replace(99,
                                        'family_unknown')

In [23]:
data4['drink_frequency'] = data4['drink_frequency'].replace(9, 
                                                    0).replace(8, 
                                                    0).replace(7,
                                                    0).replace(1, 
                                                    10).replace(2,
                                                    9).replace(3,
                                                    8).replace(4,
                                                    7).replace(5,
                                                    6)

> Save to CSV

In [24]:
data4.to_csv('inmate_clean.csv')